# Lab 7. Text Classification with DistilBERT contextual embeddings

In this Lab, we are going to re-use the code from [Lab 5](https://github.com/ckittask/tartu-nlp-2021/blob/main/labs/lab_5/lab5.ipynb). As you may remember, in Lab 5 we looked into text classification with CNN. We used the [ACL IMDB v.1 Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) that contains movie reviews classified as positive and negative. 

In this notebook, you are going to learn how to train a classifier by fine-tuning BERT with [transformers](https://github.com/huggingface/transformers) library by HuggingFace.


Transformers is NLP library that provides many general-purpose architectures like BERT GPT-2, RoBERTa etc for NLO and NLG. They have over 32+ pretrained models in 100+ languages. 

## DistilBERT - distilled version of BERT
[DistilBERT](https://arxiv.org/abs/1910.01108) is a general-purpose language representation model, which is distilled version of BERT base model. It retains almost all the language undestanding capabilities of BERT, but is smaller, faster and cheaper. It has 40% less parameters than BERT-base and runs 60% faster. 

Distillation is a technique that is used to compress a larger model into smaller. Larger model is called the teacher and the smaller one the student. 
Smaller model is trained to reproduce the behaviour (mimic the output distribution) of the teacher model. 


## Fine-tuning BERT

Transfer learning is a teqnique where we use deep learning model, which is trained on a huge dataset, is used to perform some other task. This model is called pre-trained model and we can use it as a starting point for our problem instead of starting the model from scratch. These pre-trained models do not need a labeled datasets and can later be fine-tuned on smaller labeled datasets. 

In our case, the pretrained model is DistilBERT. We can use different techniques for fine-tuning. 
1. Train the entire architecture
2. Train some layers and freeze others
3. Freeze the entire architecture

In todays lab, we will be training the entire architecture including the task-specific neural network layers. 

BERT based models have 3 inputs: 
<img src="https://miro.medium.com/proxy/0*m_kXt3uqZH9e7H4w.png">
(Image from https://morioh.com/p/84c0723ac17f) <br>
As can be seen from the above images, BERT uses special tokens: [CLS], [SEP]. [CLS] is added at the beginning of every sentence. [SEP] is added in the end of every sentence. 

In text classification task, the the final hidden state corresponding to CLS token is used as an input to the task-specific neural network layers:
<img src="https://miro.medium.com/max/1032/0*x3vhaoJdGndvZqmL.png">
Image from (https://morioh.com/p/84c0723ac17f)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from pathlib import Path
import time
import random

# Check if we are running on a CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We need to install the transformers library first, if we use Google Colab.

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.0MB/s 
     |████████████████████████████████| 901kB 50.3MB/s 
     |████████████████████████████████| 3.3MB 52.6MB/s 


From transformers, we need only the model itself, which is `DistilBertForSequenceClassification`; special optimizer that works with it, which is `AdamW`; and a pretrained tokenizer `DistilBertTokenizer` to feed our data correctly into the model. 

For the sake of saving space and performance, we use `DistilBert` in this notebook.

In [ ]:
from transformers import DistilBertForSequenceClassification, AdamW, DistilBertConfig
from transformers import DistilBertTokenizer
from transformers import get_linear_schedule_with_warmup

Downloading and unpacking the data.

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!mkdir data/
!tar -xzf aclImdb_v1.tar.gz -C data/

--2021-04-20 20:37:57--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  53.6MB/s    in 1.5s    

2021-04-20 20:37:58 (53.6 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



Initializig the tokenizer from the pretrained `distilbert-base-uncased` model.

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True, padding_side='right')

In [ ]:
print(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today."))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("This is the BERT tokenizer that we're going to use today.")))


['this', 'is', 'the', 'bert', 'token', '##izer', 'that', 'we', "'", 're', 'going', 'to', 'use', 'today', '.']
[2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012]


In [ ]:
tokenizer.encode_plus("This is the BERT tokenizer that we're going to use today.")

{'input_ids': [101, 2023, 2003, 1996, 14324, 19204, 17629, 2008, 2057, 1005, 2128, 2183, 2000, 2224, 2651, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
PAD = '[PAD]'
PAD_ID = 0
DATA_PATH = Path('data') / 'aclImdb'

batch_size = 16
validation_split = .3
shuffle_dataset = True
random_seed = 42

Here, we are going to re-use the `IMDBDataSet` class that we defined in the Lab 4. The only difference is that we get rid of pretrained word embeddings and replace a Spacy tokenizer with our pretrained `DistilBertTokenizer`.

Remember, that in order to use BERT model, our sequence must have the `[CLS]` special token in the beginning and `[SEP]` in the end of the sequence. If we are using `DistilBertTokenizer.encode()` method, we can just specify `add_special_tokens=True` to not bother with it manually.

Another important thing is that the pretrained `distilbert-base-uncased` model only accepts sequences of maximum length of 512 tokens. We set the `max_length` parameter to the `DistilBertTokenizer.encode()` method to trim them accordingly.

Last modification that we make is adding `attention_masks`. This vector is basically telling the model which characters are meaningful and which one are used for padding. To do that, we put `1` in the position of meaningful tokens and `0` in the position of paddings.

Since we already return torch tensors padded to have equal lengths in the Dataset, we don't need to define a custom `collate_fn` function as we did in the Lab 4. The default `collate_fn` function from `DataLoader` will handle the collating for us.

Last modification that we make is adding `attention_masks`. This vector is basically telling the model which characters are meaningful and which one are used for padding. To do that, we put `1` in the position of meaningful tokens and `0` in the position of paddings.

In [ ]:
class IMDBDataSet(Dataset):
    def __init__(self, pretrain_tokenizer, data_folder='.data', test=False):
        self.tokenizer = pretrain_tokenizer
        self.label_vocab = {'neg': 0, 'pos': 1}
        self.max_len = 512

        if test:
            self.data_folder = data_folder / 'test'
        else:
            self.data_folder = data_folder / 'train'
            
        self.data = []
        
        if self.data_folder.exists():
            self.load()
        else:
            raise ValueError("Data path doesn't exist!")
        
    def load(self):
        for label in ['pos', 'neg']:
            print(f'Reading {label} sentences...')
            p = self.data_folder / label
            for fname in p.glob('*.txt'):
              with open(fname, 'r', encoding='utf-8') as f:
                text = self.tokenizer.encode(f.readline(), add_special_tokens=True, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
                attention_mask = text > 0 
                attention_mask = attention_mask.squeeze().to(torch.uint8)
              torch_label = torch.tensor(self.label_vocab[label], dtype=torch.long)
              # append text, attention text and torch_label
              self.data.append((text.squeeze(), attention_mask, torch_label))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1], self.data[idx][2]

In [ ]:
torch.tensor([[1,2,3]]).squeeze()

tensor([1, 2, 3])

In [ ]:
train_data = IMDBDataSet(tokenizer, DATA_PATH)

Reading pos sentences...
Reading neg sentences...


In [ ]:
test_data = IMDBDataSet(tokenizer, DATA_PATH, test=True)

Reading pos sentences...
Reading neg sentences...


In [ ]:
train_data[0]

(tensor([  101,  2043,  2026,  2085,  2267,  2287,  2684,  2001,  1999, 23655,
          1010,  2023, 13612,  2596,  2006,  1037,  1004,  1041,  2013,  1022,
          1011,  1023,  2169,  2851,  1012,  2026, 11429,  1998,  1045,  2081,
          1037, 14790,  2008,  2057,  2876,  1005,  1056,  3335,  1037,  3371,
          1997,  4869, 26975,  1998,  2256,  4268,  2020,  2397,  2005, 23655,
          2296,  2851,  2005,  1996,  2878,  2733,  1012,  2204,  3601,  1012,
          1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  1045,  1005,
          1040,  6404,  2129,  2172,  1045,  3866,  2023,  3185,  2127,  1045,
          2288,  2041,  2026,  2214, 17550,  6100,  3728,  1012, 10805, 12413,
          2003,  2200,  8502,  1010,  2021,  2145,  3819,  2004, 10541,  1012,
          1996,  2601,  1010, 13675,  8490,  6292,  2227,  1010,  1996, 17727,
         11124,  3560, 21745, 22148,  2007,  8562,  1998, 24605,  2020,  3442,
          2013,  1996,  5530,  1997,  1996,  2338,  

In [ ]:
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
validation_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(train_data.label_vocab), output_attentions=False, output_hidden_states=False)
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The DistilBERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The DistilBERT model has 104 different named parameters.

==== Embedding Layer ====

distilbert.embeddings.word_embeddings.weight            (30522, 768)
distilbert.embeddings.position_embeddings.weight          (512, 768)
distilbert.embeddings.LayerNorm.weight                        (768,)
distilbert.embeddings.LayerNorm.bias                          (768,)
distilbert.transformer.layer.0.attention.q_lin.weight     (768, 768)

==== First Transformer ====

distilbert.transformer.layer.0.attention.q_lin.bias           (768,)
distilbert.transformer.layer.0.attention.k_lin.weight     (768, 768)
distilbert.transformer.layer.0.attention.k_lin.bias           (768,)
distilbert.transformer.layer.0.attention.v_lin.weight     (768, 768)
distilbert.transformer.layer.0.attention.v_lin.bias           (768,)
distilbert.transformer.layer.0.attention.out_lin.weight   (768, 768)
distilbert.transformer.layer.0.attention.out_lin.bias         (768,)
distilbert.transformer.layer.0.sa_layer_norm.weight      

In [ ]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

epochs = 3
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# Taken from this tutorial: https://github.com/aniruddhachoudhury/BERT-Tutorials/blob/master/Blog%202/BERT_Fine_Tuning_Sentence_Classification.ipynb
# The code was modified

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed_mins, elapsed_secs = epoch_time(t0, time.time())
            
            # Report progress.
            print(f'  Batch {step:>5,}  of  {len(train_loader):>5,}.    Elapsed: {elapsed_mins:}m {elapsed_secs:}s.')

        # Unpack this training batch from our dataloader. 
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print(f"  Average training loss: {avg_train_loss:.2f}")
    print("  Training epcoh took: {:}m {:}s".format(*epoch_time(t0, time.time())))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
            outputs = model(b_input_ids, b_input_mask)


        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}m {:}s".format(*epoch_time(t0, time.time())))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
  Batch    40  of  1,094.    Elapsed: 0m 17s.
  Batch    80  of  1,094.    Elapsed: 0m 34s.
  Batch   120  of  1,094.    Elapsed: 0m 52s.
  Batch   160  of  1,094.    Elapsed: 1m 9s.
  Batch   200  of  1,094.    Elapsed: 1m 26s.
  Batch   240  of  1,094.    Elapsed: 1m 44s.
  Batch   280  of  1,094.    Elapsed: 2m 1s.
  Batch   320  of  1,094.    Elapsed: 2m 18s.
  Batch   360  of  1,094.    Elapsed: 2m 36s.
  Batch   400  of  1,094.    Elapsed: 2m 53s.
  Batch   440  of  1,094.    Elapsed: 3m 10s.
  Batch   480  of  1,094.    Elapsed: 3m 28s.
  Batch   520  of  1,094.    Elapsed: 3m 45s.
  Batch   560  of  1,094.    Elapsed: 4m 2s.
  Batch   600  of  1,094.    Elapsed: 4m 20s.
  Batch   640  of  1,094.    Elapsed: 4m 37s.
  Batch   680  of  1,094.    Elapsed: 4m 54s.
  Batch   720  of  1,094.    Elapsed: 5m 12s.
  Batch   760  of  1,094.    Elapsed: 5m 29s.
  Batch   800  of  1,094.    Elapsed: 5m 46s.
  Batch   840  of  1,094.    Elapsed: 6m

In [ ]:
print("")
print("Running Testing...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables 
test_loss, test_accuracy = 0, 0
nb_test_steps, nb_test_examples = 0, 0

# Evaluate data for one epoch
for batch in test_loader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
        outputs = model(b_input_ids, b_input_mask)


    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Calculate the accuracy for this batch of test sentences.
    tmp_test_accuracy = flat_accuracy(logits, label_ids)
    
    # Accumulate the total accuracy.
    test_accuracy += tmp_test_accuracy

    # Track the number of batches
    nb_test_steps += 1

# Report the final accuracy for this test run.
print("  Accuracy: {0:.2f}".format(test_accuracy/nb_test_steps))
print("  Testing took: {:}m {:}s".format(*epoch_time(t0, time.time())))


Running Testing...
  Accuracy: 0.93
  Testing took: 3m 38s


In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
model = DistilBertForSequenceClassification.from_pretrained(output_dir)
tokenizer = DistilBertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       